# Intro

This notebook gives the design decision made to decide on the sizing of the single port SRAM cell

# Setup

## Imports

In [1]:
import os
from os.path import realpath
from typing import Optional, overload, cast

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
import ipywidgets as widgets

import numpy as np
from textwrap import dedent
import matplotlib.pyplot as plt

from pdkmaster.technology import primitive as _prm

from c4m.flexmem import SRAMCellSP6T
from c4m.pdk import sky130
prims = sky130.tech.primitives

## PySpice

Ngspice is used together with PySpice for the simulation part of this workbook. In order to increase parsing of the model files for the sky130 PDK a version >= 35 of ngspice has to be used. Also the provided `.spiceinit` file in the running directory is needed to get this speed up.

PySpice uses the `NGPSICE_LIBRARY_PATH` environment variable to find the ngspice library so you may need to set that shell environment before starting the notebook server if it fails to find it on it's own. If your distribution ngspice version is < 35 you can compile your own version and then set the `NGSPICE_LIBRARY_PATH` to the proper path.

More specifically on debian 11 the system ngspice version is 34 but you can install the library from unstable using [apt pinning](https://jaqque.sbih.org/kplug/apt-pinning.html) which at the time of writing has version 39:

    % sudo apt install -y -t unstable libngspice0
    % # Bash syntax, needs conversion for csh etc.
    % export NGSPICE_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu/libngspice.so.0


## Sky130 SRAM cell class

In [2]:
class SRAMCellSP6T_Sky130(SRAMCellSP6T):
    def __init__(self, *,
        name: str,
        mos_pullup: _prm.MOSFET=cast(_prm.MOSFET, prims.pfet_01v8),
        l_pullup: float, w_pullup: float,
        mos_pulldown: _prm.MOSFET=cast(_prm.MOSFET, prims.nfet_01v8_sc),
        l_pulldown: float, w_pulldown: float,
        mos_passgate: _prm.MOSFET=cast(_prm.MOSFET, prims.nfet_01v8_sc),
        l_passgate: float, w_passgate: float,
    ):
        super().__init__(
            cktfab=sky130.cktfab, pyspicefab=sky130.pyspicefab, name=name,
            mos_pullup=mos_pullup, l_pullup=l_pullup, w_pullup=w_pullup,
            mos_pulldown=mos_pulldown, l_pulldown=l_pulldown, w_pulldown=w_pulldown,
            mos_passgate=mos_passgate, l_passgate=l_passgate, w_passgate=w_passgate,
            nom_vdd=1.8, nom_corner="logic_ff", nom_temperature=25,
        )

# SRAM design

The classic 6 transistor single port SRAM cells consists of two back-to-back invertors (e.g a latch) with two nmos access transistors. The pmos of the invertors is called the pull-up transistor, the nmos the pull-down transistor and the access transistor are also called pass gates.

Designing an SRAM cell consists of determining l and w of the transistors. This will involve a trade-off between speed, area, data retention, power consumption and write-ability. Different test benches are defined for deriving metrics to allow to make the trade-off.

Several metrics are defined in order to allow to quantify the trade-off being made:
- Read Current (speed)
- Leakage Current (static power consumption)
- Static Noise Margin (non-destructive read)
- Write Trip Point (write-ability)
- Capacitive load on bit line (speed & dynamic power consumption)
- Capacitive load on word line (second order dynamic power consumption)

Speed is determined both by the read current and the capacitive load of the bitline; the read current says how fast it can decharge the bit line. The capacitive load on the bit line is determined by the w$_{PG}$ and of the parasitic capacitance of the  metal line for the bit line. The R of this bit line may also be contributing to the delay for large bit lines; e.g. with SRAM blocks with high number of rows.

# Manual exploration

First one can view manually the impact of the cell design on the different metrics.


In [3]:
@widgets.interact
def sram_metrics(
    corner=widgets.Dropdown(
        value="logic_tt", options=["logic_tt", "logic_ff", "logic_ss", "logic_sf", "logic_fs"],
        description="corner",
    ),
    temp=widgets.FloatText(value=27, description="temp. [℃]"),
    vdd=widgets.FloatText(value=1.8, description="Vdd [V]"),
    l_pu=widgets.FloatText(value=0.15, description="l_PU [µm]"),
    w_pu=widgets.FloatText(value=0.42, description="w_PU [µm]"),
    l_pd=widgets.FloatText(value=0.15, description="l_PD [µm]"),
    w_pd=widgets.FloatText(value=0.42, description="w_PD [µm]"),
    l_pg=widgets.FloatText(value=0.15, description="l_PG [µm]"),
    w_pg=widgets.FloatText(value=0.42, description="w_PG [µm]"),
    simulate=widgets.Checkbox(value=True, description="Simulate")
):
    if not simulate:
        print("Simulation disabled")
        return

    cell = SRAMCellSP6T_Sky130(
        name="sramsp6t",
        mos_pullup=prims.pfet_01v8, l_pullup=l_pu, w_pullup=w_pu,
        mos_pulldown=prims.nfet_01v8, l_pulldown=l_pd, w_pulldown=w_pd,
        mos_passgate=prims.nfet_01v8_sc, l_passgate=l_pg, w_passgate=w_pg,
    )
    print("1/6")
    Iread = cell.Iread(corner=corner, vdd=vdd, temperature=temp)
    print("2/6")
    Ileak = cell.Ileak(corner=corner, vdd=vdd, temperature=temp)
    print("3/6")
    Pstatic = Ileak*vdd
    print("4/6")
    SNMhold = cell.SNM(read=False, corner=corner, vdd=vdd, temperature=temp)
    print("5/6")
    SNMread = cell.SNM(read=True, corner=corner, vdd=vdd, temperature=temp)
    print("6/6")
    WTP = cell.WTP(corner=corner, vdd=vdd, temperature=temp)
    printmd(dedent("""
        | metric | value |
        |-:|:-:|
        | I_read | {:.3f}mA |
        | I_leak | {:.3f}nA |
        | P_static | {:.3f}mW/Mbit |
        | SNM_hold | {:.1f}mV |
        | SNM_read | {:.1f}mV |
        | WTP | {:.1f}mV |
    """.format(
        Iread*1e3, Ileak*1e9, Pstatic*1e9,
        SNMhold*1e3, SNMread*1e3, WTP*1e3,
    )))

interactive(children=(Dropdown(description='corner', options=('logic_tt', 'logic_ff', 'logic_ss', 'logic_sf', …

# Worst case corners

In this chapter the worst case corner will be determined for each of the metrics.

Values considered:

* corners: logic_ss, logic_ff, logic_sf, logic_fs
* vdd: 1.62...1.98V
* temperature: -25...85°C


In [4]:
testcell = SRAMCellSP6T_Sky130(
    name="sp6t",
    mos_pullup=prims.pfet_01v8, l_pullup=0.15, w_pullup=0.42,
    mos_pulldown=prims.nfet_01v8_sc, l_pulldown=0.15, w_pulldown=0.42,
    mos_passgate=prims.nfet_01v8_sc, l_passgate=0.15, w_passgate=0.42,
)


## SNMread

In [5]:
temps = [-25, 25, 85]
corners = ["logic_tt", "logic_ss", "logic_ff", "logic_sf", "logic_fs"]

n_temps = len(temps)
n_corners = len(corners)

SNMread = np.zeros((n_corners, n_temps))
vdd = 1.62
s = f"___SNM_read [mV] (vdd={vdd:.2f}V)___\n\n"
s += "| | " + " | ".join([str(temp) for temp in temps]) + " |\n"
s += "|-:|" + n_temps*":-:|" + "\n"
for i_corner, corner in enumerate(corners):
    print(f"{i_corner + 1}/{len(corners)}")
    s += "| __" + corner + "__ |"
    for i_temp, temp in enumerate(temps):
        SNM = testcell.SNM(read=True, corner=corner, vdd=vdd, temperature=temp)
        SNMread[i_corner, i_temp] = SNM
        s+= " {:.1f} |".format(SNM*1e3)
    s += "\n"

printmd(s)

1/5
2/5
3/5
4/5
5/5


___SNM_read [mV] (vdd=1.62V)___

| | -25 | 25 | 85 |
|-:|:-:|:-:|:-:|
| __logic_tt__ | 663.0 | 655.9 | 645.5 |
| __logic_ss__ | 692.9 | 694.5 | 690.6 |
| __logic_ff__ | 604.4 | 592.6 | 580.6 |
| __logic_sf__ | 523.5 | 505.6 | 481.9 |
| __logic_fs__ | 774.8 | 771.3 | 762.5 |


## WTP

In [6]:
temps = [-25, 25, 85]
corners = ["logic_tt", "logic_ss", "logic_ff", "logic_sf", "logic_fs"]

n_temps = len(temps)
n_corners = len(corners)

WTP = np.zeros((n_corners, n_temps))
vdd = 1.62
s = f"___WTP [mV] (vdd={vdd:.2f}V)___\n\n"
s += "| | " + " | ".join([str(temp) for temp in temps]) + " |\n"
s += "|-:|" + n_temps*":-:|" + "\n"
for i_corner, corner in enumerate(corners):
    print(f"{i_corner + 1}/{len(corners)}")
    s += "| __" + corner + "__ |"
    for i_temp, temp in enumerate(temps):
        v = testcell.WTP(corner=corner, vdd=vdd, temperature=temp)
        WTP[i_corner, i_temp] = v
        s+= f" {v*1e3:.1f} |"
    s += "\n"

printmd(s)

1/5
2/5
3/5
4/5
5/5


___WTP [mV] (vdd=1.62V)___

| | -25 | 25 | 85 |
|-:|:-:|:-:|:-:|
| __logic_tt__ | 590.0 | 586.8 | 580.4 |
| __logic_ss__ | 528.8 | 529.1 | 527.1 |
| __logic_ff__ | 661.3 | 648.5 | 633.0 |
| __logic_sf__ | 803.8 | 809.0 | 811.1 |
| __logic_fs__ | 363.2 | 341.7 | 317.7 |


- Read current is mainly determined by the drive strength of the pass gate and the pull down. This is lowest for 'SS' process, lowest Vdd and highest temperature
- Highest leakage/static power consumption is seen for logic_ff process, highest Vdd and highest temperature
- SNM_read is lowest for the stronger nmos and weaker pmos (logic_sf corner), lowest Vdd and the highest temperature
- WTP is lowest for weaker nmos and stronger pmos (logic_fs corner), lowest Vdd; and higher temperature does not much influence on WTP

In [7]:
class SRAMCellSP6T_Sky130_WC(SRAMCellSP6T_Sky130):
    Vdd_nom = 1.8
    @property
    def Vdd_low(self):
        return 0.9*self.Vdd_nom
    @property
    def Vdd_high(self):
        return 1.1*self.Vdd_nom
    
    temp_nom = 25
    temp_low = -25
    temp_high = 85

    @property
    def Iread_corner(self):
        return {"vdd": self.Vdd_low, "corner": "logic_ss", "temperature": self.temp_high}
    def Iread(self, *, vwl: Optional[float]=None, vpre: Optional[float]=None):
        return super().Iread(vwl=vwl, vpre=vpre, **self.Iread_corner)

    @property
    def Ileak_corner(self):
        return {"vdd": self.Vdd_high, "corner": "logic_ff", "temperature": self.temp_high}
    def Ileak(self, *, vpre: Optional[float]=None, floatbl: bool=False):
        return super().Ileak(vpre=vpre, floatbl=floatbl, **self.Ileak_corner)
    
    @property
    def SNM_corner(self):
        return {"vdd": self.Vdd_low, "corner": "logic_sf", "temperature": self.temp_high}
    @overload
    def SNM(self, *, read: bool=True, vwl: None=None, vpre: Optional[float]=None) -> float:
        ...
    @overload
    def SNM(self, *, read: None, vwl: float, vpre: Optional[float]=None) -> float:
        ...
    def SNM(self, *, read: Optional[bool]=True, vwl: Optional[float]=None, vpre: Optional[float]=None) -> float:
        return super().SNM(read=read, vwl=vwl, vpre=vpre, **self.SNM_corner)
    
    @property
    def WTP_corner(self):
        return {"vdd": self.Vdd_low, "corner": "logic_fs", "temperature": self.temp_high}
    def WTP(self, vwl: Optional[float]=None, vpre: Optional[float]=None):
        return super().WTP(vwl=vwl, vpre=vpre, **self.WTP_corner)

In [8]:
@widgets.interact
def sram_metrics(
    l_pu=widgets.FloatText(value=0.15, description="l_PU [µm]"),
    w_pu=widgets.FloatText(value=0.42, description="w_PU [µm]"),
    l_pd=widgets.FloatText(value=0.15, description="l_PD [µm]"),
    w_pd=widgets.FloatText(value=0.36, description="w_PD [µm]"),
    l_pg=widgets.FloatText(value=0.15, description="l_PG [µm]"),
    w_pg=widgets.FloatText(value=0.36, description="w_PG [µm]"),
    simulate=widgets.Checkbox(value=True, description="Simulate")
):
    if not simulate:
        print("Simulation disabled")

    cell = SRAMCellSP6T_Sky130_WC(
        name="sramsp6t",
        l_pullup=l_pu, w_pullup=w_pu,
        l_pulldown=l_pd, w_pulldown=w_pd,
        l_passgate=l_pg, w_passgate=w_pg,
    )
    
    print("1/6")
    Iread = cell.Iread()
    print("2/6")
    Ileak = cell.Ileak()
    print("3/6")
    Pstatic = Ileak*cell.Ileak_corner["vdd"]
    print("4/6")
    SNMhold = cell.SNM(read=False)
    print("5/6")
    SNMread = cell.SNM(read=True)
    print("6/6")
    WTP = cell.WTP()
    printmd(dedent(f"""
        | metric | value | corner | V_dd [V] | temp [℃] |
        |-:|:-:|:-:|:-:|:-:|
        | I_read | {Iread*1e6:.1f}µA | {cell.Iread_corner["corner"]} | {cell.Iread_corner["vdd"]:.2f} | {cell.Iread_corner["temperature"]:d} |
        | I_leak | {Ileak*1e9:.2f}nA | {cell.Ileak_corner["corner"]} | {cell.Ileak_corner["vdd"]:.2f} | {cell.Ileak_corner["temperature"]:d} |
        | P_static | {Pstatic*1e9:.1f}mW/Mbit | {cell.Ileak_corner["corner"]} | {cell.Ileak_corner["vdd"]:.2f} | {cell.Ileak_corner["temperature"]:d} |
        | SNM_hold | {SNMhold*1e3:.1f}mV | {cell.SNM_corner["corner"]} | {cell.SNM_corner["vdd"]:.2f} | {cell.SNM_corner["temperature"]:d} |
        | SNM_read | {SNMread*1e3:.1f}mV | {cell.SNM_corner["corner"]} | {cell.SNM_corner["vdd"]:.2f} | {cell.SNM_corner["temperature"]:d} |
        | WTP | {WTP*1e3:.1f}mV | {cell.WTP_corner["corner"]} | {cell.WTP_corner["vdd"]:.2f} | {cell.WTP_corner["temperature"]:d} |
    """))

interactive(children=(FloatText(value=0.15, description='l_PU [µm]'), FloatText(value=0.42, description='w_PU …

The minimum DRC compliant w of the transistor are relatively big for Sky130. This results in a stable and writeable cell with dimensions of all 6 transistor using minimum w of 0.42µm and l of 0.15µm.
The NMOS models also allow min_w of 0.36µm resulting still in good performance metrics.